In [1]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.feature_selection import RFE
import operator
import seaborn as sn
from scipy import stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_predict
from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB
import pandasql as ps
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/TRAIN_CLEARED2.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11028 entries, 0 to 11027
Data columns (total 48 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   NewName                                      11028 non-null  object 
 1   ClubInvolved                                 11028 non-null  object 
 2   League                                       11028 non-null  object 
 3   MainPosition                                 11028 non-null  object 
 4   LeagueInvolved                               11028 non-null  object 
 5   CorrMarketValue                              11028 non-null  float64
 6   Country                                      11028 non-null  object 
 7   Club                                         11028 non-null  object 
 8   Position                                     11028 non-null  object 
 9   Foot                                         11028 non-null  object 
 10

In [4]:
df = df.drop(['NewName', 'ClubInvolved', 'Club', 'ClubAge', 'ClubNumPlayersUsedPerMatch',\
     'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch', 'ClubPointsPerMatch', 'Fee'], axis = 1)

In [5]:
df['target'] = 0
df.loc[df.League == 'English Premier League', 'target'] = 1

In [6]:
df = df.drop('League', axis=1)

In [7]:
df['target'].value_counts()

0    9675
1    1353
Name: target, dtype: int64

In [8]:
#set feature list and target
feature_list = list(df.columns.values)
feature_list.remove('target')
target = 'target'

# define dataset
X = df[feature_list]
y = df[target]

In [9]:
import category_encoders as ce
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

In [10]:
#Impute missing values
imputer = IterativeImputer()
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

In [11]:
#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

In [12]:
from sklearn.model_selection import RepeatedStratifiedKFold
# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 11.959732663631438
score_time 0.14581093788146973
test_acc 0.9027934868277079
test_recall 0.37961601307189546
test_auc 0.885934136684521
test_precison 0.6900263739126664
test_f1 0.48873562726693304


In [13]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 11.959732663631438
score_time 0.14581093788146973
test_acc 0.9027934868277079
test_recall 0.37961601307189546
test_auc 0.885934136684521
test_precison 0.6900263739126664
test_f1 0.48873562726693304


Feature engineering

In [14]:
column_list = X_imp_st.columns
interactions = list(combinations(column_list, 2))

In [15]:
interaction_dict = {}
for interaction in interactions:
   X_train_int = X_imp_st
   X_train_int['int'] = X_train_int[interaction[0]] * X_train_int[interaction[1]]
   lr3 = LinearRegression()
   lr3.fit(X_train_int, y)
   interaction_dict[lr3.score(X_train_int, y)] = interaction

In [16]:
top_5 = sorted(interaction_dict.keys(), reverse = True)[:5]
for interaction in top_5:
   print(interaction_dict[interaction])

('LeagueInvolved', 'MinutesPlayedPer90')
('CorrMarketValue', 'IntReputation')
('CorrMarketValue', 'Reactions')
('EN_PL', 'MinutesPlayedPer90')
('CorrMarketValue', 'Passing_main')


In [30]:
#X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
# X_imp_st = X_imp_st.drop('LeagueInvolved_MinutesPlayedPer90', axis = 1)
# X_imp_st['CorrMarketValue_IntReputation'] = X_imp_st['CorrMarketValue'] * X_imp_st['IntReputation']
# X_imp_st = X_imp_st.drop('CorrMarketValue_IntReputation', axis = 1)
X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
# X_imp_st['EN_PL_MinutesPlayedPer90'] = X_imp_st['EN_PL'] * X_imp_st['MinutesPlayedPer90']
# X_imp_st = X_imp_st.drop('EN_PL_MinutesPlayedPer90', axis = 1)
# X_imp_st['CorrMarketValue_Passing_main'] = X_imp_st['CorrMarketValue'] * X_imp_st['Passing_main']
# X_imp_st = X_imp_st.drop('CorrMarketValue_Passing_main', axis = 1)

In [41]:
import warnings
warnings.filterwarnings(action="ignore", message=r".*Use subset.*of np.ndarray is not recommended")

In [35]:
X_imp_st = X_imp_st.drop('int', axis = 1)

In [36]:
from sklearn.model_selection import RepeatedStratifiedKFold
# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 13.024199318885803
score_time 0.06134563684463501
test_acc 0.9017502176048492
test_recall 0.38363562091503267
test_auc 0.8858219130066013
test_precison 0.6774685982987132
test_f1 0.48905180147053856


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [37]:
X_imp_st.columns

Index(['MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country',
       'Position', 'Foot', 'IntReputation', 'Heading_accuracy', 'Reactions',
       'Balance', 'Jumping', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA',
       'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'ClubInvolvedAge',
       'ClubInvolvedCleanSheetsRate', 'MinutesPlayedPer90', 'ContractLeft',
       'Pace_main', 'Shooting_main', 'Defending_main', 'Passing_main',
       'Physical_main', 'Age', 'ClubInvolvedPointsPerMatch',
       'ClubInvolvedNumPlayersUsedPerMatch',
       'ClubInvolvedGoalPerShotOnTargetPerMatch',
       'ClubInvolvedShotOnTargetAgainstUsedPerMatch', 'GoalsPerMinutesPlayed',
       'AssistsPerMinutesPlayed', 'YellowCardsPerMinutesPlayed',
       'RedCardsPerMinutesPlayed', 'CorrMarketValue_Reactions'],
      dtype='object')

In [38]:
column_list = X_imp_st.columns
interactions = list(combinations(column_list, 2))

interaction_dict = {}
for interaction in interactions:
   X_train_int = X_imp_st
   X_train_int['int'] = X_train_int[interaction[0]] / X_train_int[interaction[1]]
   lr3 = LinearRegression()
   lr3.fit(X_train_int, y)
   interaction_dict[lr3.score(X_train_int, y)] = interaction

top_5 = sorted(interaction_dict.keys(), reverse = True)[:5]
for interaction in top_5:
   print(interaction_dict[interaction])

('Reactions', 'EN_PL')
('IntReputation', 'EN_PL')
('Country', 'IntReputation')
('MainPosition', 'GE_BUNDESLIGA')
('Foot', 'GE_BUNDESLIGA')


In [39]:
X_imp_st =X_imp_st.drop('int', axis =1)

In [54]:
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
# X_imp_st['Country_IntReputation'] = X_imp_st['Country'] / X_imp_st['IntReputation']
# X_impt_st = X_imp_st.drop('Country_IntReputation', axis = 1)
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']
#X_imp_st['Foot_GE_BUNDESLIGA'] = X_imp_st['Foot'] / X_imp_st['GE_BUNDESLIGA']
#X_impt_st = X_imp_st.drop('Foot_GE_BUNDESLIGA', axis = 1)

In [55]:
from sklearn.model_selection import RepeatedStratifiedKFold
# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 11.04513920545578
score_time 0.06010171175003052
test_acc 0.9027931988817824
test_recall 0.38845860566448803
test_auc 0.8839635268316164
test_precison 0.6838016280820549
test_f1 0.4947498574896557


In [56]:
X_imp_st.columns

Index(['MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country',
       'Position', 'Foot', 'IntReputation', 'Heading_accuracy', 'Reactions',
       'Balance', 'Jumping', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA',
       'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'ClubInvolvedAge',
       'ClubInvolvedCleanSheetsRate', 'MinutesPlayedPer90', 'ContractLeft',
       'Pace_main', 'Shooting_main', 'Defending_main', 'Passing_main',
       'Physical_main', 'Age', 'ClubInvolvedPointsPerMatch',
       'ClubInvolvedNumPlayersUsedPerMatch',
       'ClubInvolvedGoalPerShotOnTargetPerMatch',
       'ClubInvolvedShotOnTargetAgainstUsedPerMatch', 'GoalsPerMinutesPlayed',
       'AssistsPerMinutesPlayed', 'YellowCardsPerMinutesPlayed',
       'RedCardsPerMinutesPlayed', 'CorrMarketValue_Reactions',
       'Reactions_EN_PL', 'Country_IntReputation', 'IntReputation_EN_PL',
       'MainPosition_GE_BUNDESLIGA', 'Foot_GE_BUNDESLIGA'],
      dtype='object')

In [57]:
column_list = X_imp_st.columns
interactions = list(combinations(column_list, 2))

interaction_dict = {}
for interaction in interactions:
   X_train_int = X_imp_st
   X_train_int['int'] = X_train_int[interaction[0]] + X_train_int[interaction[1]]
   lr3 = LinearRegression()
   lr3.fit(X_train_int, y)
   interaction_dict[lr3.score(X_train_int, y)] = interaction

top_5 = sorted(interaction_dict.keys(), reverse = True)[:5]
for interaction in top_5:
   print(interaction_dict[interaction])

('IntReputation', 'Reactions_EN_PL')
('Eigv', 'CorrMarketValue_Reactions')
('Country', 'FR_LIGUE_1')
('DEGREE', 'Shooting_main')
('IntReputation', 'ClubInvolvedAge')


In [75]:
X_imp_st = X_imp_st.drop('int', axis=1)

In [78]:
#X_imp_st ['IntReputation_Reactions_EN_PL'] = X_imp_st['IntReputation'] + X_imp_st['Reactions_EN_PL']
#X_imp_st = X_imp_st.drop('IntReputation_Reactions_EN_PL', axis = 1)
#X_imp_st['Eigv_CorrMarketValue_Reactions'] = X_imp_st['Eigv'] + X_imp_st['CorrMarketValue_Reactions']
#X_imp_st = X_imp_st.drop('Eigv_CorrMarketValue_Reactions', axis = 1)
# X_imp_st['Country_FR_LIGUE_1'] = X_imp_st['Country'] + X_imp_st['FR_LIGUE_1']
# X_imp_st = X_imp_st.drop('Country_FR_LIGUE_1', axis = 1)
# X_imp_st['DEGREE_Shooting_main'] = X_imp_st['DEGREE'] + X_imp_st['Shooting_main']
# X_imp_st = X_imp_st.drop('DEGREE_Shooting_main', axis = 1)
#X_imp_st['IntReputation_ClubInvolvedAge'] = X_imp_st['IntReputation'] + X_imp_st['ClubInvolvedAge']
X_imp_st = X_imp_st.drop('IntReputation_ClubInvolvedAge', axis = 1)

In [79]:
from sklearn.model_selection import RepeatedStratifiedKFold
# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 9.232471776008605
score_time 0.04809315204620361
test_acc 0.9027931988817824
test_recall 0.38845860566448803
test_auc 0.8839635268316164
test_precison 0.6838016280820549
test_f1 0.4947498574896557


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

In [80]:
X_imp_st.columns

Index(['MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country',
       'Position', 'Foot', 'IntReputation', 'Heading_accuracy', 'Reactions',
       'Balance', 'Jumping', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA',
       'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'ClubInvolvedAge',
       'ClubInvolvedCleanSheetsRate', 'MinutesPlayedPer90', 'ContractLeft',
       'Pace_main', 'Shooting_main', 'Defending_main', 'Passing_main',
       'Physical_main', 'Age', 'ClubInvolvedPointsPerMatch',
       'ClubInvolvedNumPlayersUsedPerMatch',
       'ClubInvolvedGoalPerShotOnTargetPerMatch',
       'ClubInvolvedShotOnTargetAgainstUsedPerMatch', 'GoalsPerMinutesPlayed',
       'AssistsPerMinutesPlayed', 'YellowCardsPerMinutesPlayed',
       'RedCardsPerMinutesPlayed', 'CorrMarketValue_Reactions',
       'Reactions_EN_PL', 'Country_IntReputation', 'IntReputation_EN_PL',
       'MainPosition_GE_BUNDESLIGA', 'Foot_GE_BUNDESLIGA'],
      dtype='object')